In [0]:
%pip freeze

###Download extracted entities dataset, run this cell if you want to go straight to entity linkage without extracting the Named Entities yourself.If you do this, you do not need to install the NER models and import them.

In [0]:
import spacy
#import scispacy
#import swifter
import pandas as pd
from spacy import displacy
#import en_core_sci_scibert
import en_core_sci_lg
import en_ner_bc5cdr_md
#import en_core_sci_sm
"""
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
from scispacy.abbreviation import AbbreviationDetector
"""
from scispacy.linking import EntityLinker
from collections import OrderedDict,Counter
from pprint import pprint
from tqdm import tqdm
tqdm.pandas()

install -r /Volumes/edav_dev_cdh/cdh_ml/metadata_compute/requirements_scispacy_lg.txt
%pip install /Volumes/edav_dev_cdh/cdh_ml/metadata_compute/packages/wheels/en_ner_bc5cdr_md-0.5.4-py3-none-any.whl
#import en_core_sci_md
# import en_ner_bc5cdr_md
# import en_ner_jnlpba_md
# import en_ner_craft_md
# import en_ner_bionlp13cg_md
dbutils.library.restartPython()

In [0]:
#en_core_sci_lg.load()


### The sample text here is from 
###Denison MR. CORONAVIRUS RESEARCH: KEYS TO DIAGNOSIS, TREATMENT, AND PREVENTION OF SARS. In: Institute of Medicine (US) Forum on Microbial Threats; Knobler S, Mahmoud A, Lemon S, et al., editors. Learning from SARS: Preparing for the Next Disease Outbreak: Workshop Summary. Washington (DC): National Academies Press (US); 2004. Available from: https://www.ncbi.nlm.nih.gov/books/NBK92477/

In [0]:
note=spark.table("edav_dev_cdh.cdh_mimic_ra.note_clean").limit(100)
display(note)

In [0]:
nlp = en_ner_bc5cdr_md.load()


@pandas_udf(ArrayType(StringType()))
def entities(document: pd.Series) -> pd.Series:
#def display_entities(nlp,document):
    """ A function that returns a tuple of displacy image of named or unnamed word entities and
        a set of unique entities recognized based on scispacy model in use
        Args: 
            model: A pretrained model from spaCy or ScispaCy
            document: text data to be analysed"""
    doc = nlp(document)
#    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = set([(ent.text, ent.label_,ent.start,ent.end,) for ent in doc.ents])
 #   return  displacy_image, entity_and_label
    return entity_and_label

In [0]:
bc5dr_entities= display_entities(en_ner_bc5cdr_md,sample_text)

In [0]:
print(bc5dr_entities[1])

In [0]:
ner_columns=['lexical_variance','nlp_category','offset','end',]

In [0]:
bc5dr_entities_dataframe = pd.DataFrame(bc5dr_entities[1],columns=ner_columns)  #save returned values of entities and their labels in a pandas dataframe
bc5dr_entities_dataframe['nlp_category'] = 'bc5cdr'  #include a column with constant value of NER model
bc5dr_entities_dataframe

entities_and_label_from_4_NER_model_dataframe = pd.concat([bc5dr_entities_dataframe,bionlp13cg_entities_dataframe,craft_entities_dataframe,jnlpa_entities_dataframe])
#Concatenate all pandas dataframe into one.
entities_and_label_from_4_NER_model_dataframe.to_csv('entities_and_label_from_4_scispacy_NER_models.csv', index=False) #Save dataframe to csv
entities_and_label_from_4_NER_model_dataframe.info()

```python
#### https://spacy.io/api/entitylinker/
# Construction via add_pipe with default model  
entity_linker = nlp.add_pipe("entity_linker")  

# Construction via add_pipe with custom model  
config = {"model": {"@architectures": "my_el.v1"}}  
entity_linker = nlp.add_pipe("entity_linker", config=config)  

# Construction from class  
from spacy.pipeline import EntityLinker  
entity_linker = EntityLinker(nlp.vocab, model)  
```
https://github.com/allenai/scispacy/issues/237

```python
from scispacy.candidate_generation import DEFAULT_PATHS, DEFAULT_KNOWLEDGE_BASES
from scispacy.candidate_generation import (
    CandidateGenerator,
    LinkerPaths
)

CustomLinkerPaths_2020AA = LinkerPaths(
    ann_index="path/to/nmslib_index.bin",
    tfidf_vectorizer="path/to//nmslib_index.bin",
    tfidf_vectors="path/to/tfidf_vectorizer.joblib",
    concept_aliases_list="path/to/concept_aliases.json",
)
class UMLS2020KnowledgeBase(KnowledgeBase):
    def __init__(
        self,
        file_path: str = "path/to/2020AA.json",
    ):
        super().__init__(file_path)


# Admittedly this is a bit of a hack, because we are mutating a global object.
# However, it's just a kind of registry, so maybe it's ok.
DEFAULT_PATHS["umls2020"] = CustomLinkerPaths_2020AA
DEFAULT_KNOWLEDGE_BASES["umls2020"] = UMLS2020KnowledgeBase

linker = CandidateGenerator(name="umls2020")
```

def display_entities(model,document):
    """ A function that returns a tuple of displacy image of named or unnamed word entities and
        a set of unique entities recognized based on scispacy model in use
        Args: 
            model: A pretrained model from spaCy or ScispaCy
            document: text data to be analysed"""
    nlp = model.load()
    doc = nlp(document)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = set([(X.text, X.label_) for X in doc.ents])
    return  displacy_image, entity_and_label

In [0]:
from scispacy.candidate_generation import DEFAULT_PATHS, DEFAULT_KNOWLEDGE_BASES, CandidateGenerator, LinkerPaths
from scispacy.linking_utils import KnowledgeBase
basePath="/Volumes/edav_dev_cdh/cdh_ml/metadata_data/linker/umls" #/linker/umls"

UmlsLinkerPaths_2024AA = LinkerPaths(
ann_index=f"{basePath}/nmslib_index.bin",  # noqa
tfidf_vectorizer=f"{basePath}/tfidf_vectorizer.joblib",  # noqa
tfidf_vectors=f"{basePath}/tfidf_vectors_sparse.npz",  # noqa
concept_aliases_list=f"{basePath}/concept_aliases.json",  # noqa
)
print(UmlsLinkerPaths_2024AA)

class UMLS2024KnowledgeBase(KnowledgeBase):
    def __init__(
        self,
        file_path: str = f"/Volumes/edav_dev_cdh/cdh_ml/metadata_data/jsonl/umls_kb.jsonl",
    ):
        super().__init__(file_path)

# Admittedly this is a bit of a hack, because we are mutating a global object.
# However, it's just a kind of registry, so maybe it's ok.
DEFAULT_PATHS["umls2024"] = UmlsLinkerPaths_2024AA
DEFAULT_KNOWLEDGE_BASES["umls2024"] = UMLS2024KnowledgeBase

linker = CandidateGenerator(name="umls2024")

In [0]:
###from scispacy.candidate_generation import CandidateGenerator
#with tempfile.TemporaryDirectory() as dir_name:
#    umls_concept_aliases, tfidf_vectorizer, ann_index = create_tfidf_ann_index(dir_name, umls_fixture)
####cdh_candidate_generator = CandidateGenerator(ann_index, tfidf_vectorizer, tfidf_vectors, concept_aliases_list)

##self.linker = EntityLinker(candidate_generator=candidate_generator, filter_for_definitions=False)

# https://github.com/allenai/scispacy/issues/237
#def cdh_entity_linker(candidate_generator_name,document):
""" A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
#e than 2 entity matches
#### entity_linker=EntityLinker(k = 10,max_entities_per_mention = 2,candidate_generator=candidate_generator_name, filter_for_definitions=False, )
nlp = en_ner_bc5dr_md.load()
nlp.add_pipe("scispacy_linker")
doc = nlp(sample_text)
#name=linker_name) 
try:
    entity = doc.ents[0]
except IndexError:
    entity = 'Nan'
entity_details = []
entity_details.append(entity)
try:
    for linker_ent in entity._.kb_ents:
        Concept_Id, Score = linker_ent
        entity_details.append('Entity_Matching_Score :{}'.format(Score))
        entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
except AttributeError:
    pass
#    return entity_details

In [0]:
#dh_entity_linker(cdh_candidate_generator,sample_text)

In [0]:
nlp=spacy.load("en_ner_bc5dr_md")
nlp.pipe_names
nlp.add_pipe(
    "scispacy_linker", config={"resolve_abbreviations": False, "linker_name": "umls2024"}
)
linker = nlp.get_pipe("scispacy_linker")


In [0]:
from collections import defaultdict  # Uncommented to fix the error

doc = nlp(sample_text)
ents_dict=defaultdict(list)

for entity in doc.ents:
    ents_dict[entity.label_].append(entity.text)  # Corrected from ent._label_ to entity.label_
    print(ents_dict[entity.label_])
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            print(linker_ent)
            #Concept_Id, Score = linker_ent
            #entity_details.append('Entity_Matching_Score :{}'.format(Score))
            #entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
            print(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        print("error")
        pass

In [0]:
entity_details

###Test the entity_linker function with the 4 sciSpacy knowledge bases "umls","mesh","go","hpo". The function will return 2 entities and their scores as it relates to the Knowledge base. As at the time of this writing the "rxnorm" knowledge base returned KeyError, I will investigate further on why this is happening

In [0]:
test1 = 'fever'

In [0]:
entity_linker('umls',test1)

In [0]:
entity_linker('go',test1)

In [0]:
entity_linker('hpo',test1)

In [0]:
entity_linker('mesh',test1)

In [0]:
entity_linker('rxnorm',test1)

In [0]:
#linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='umls')  #parameters are tunable,so it can be set to return more than 2 entity matches
#nlp = en_core_sci_md.load()
#nlp.add_pipe("umls")
def umls_entity_linker(document,linker_name="umls2024"):
    """ A function that accepts and document and returns the entity link details"""
    linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    nlp = en_ner_bc5dr_md.load()
    nlp.add_pipe(linker_name)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [0]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='mesh')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_ner_bc5dr_md.load()
nlp.add_pipe(linker)
def mesh_entity_linker(document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [0]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='go')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_ner_bc5dr_md.load()
nlp.add_pipe(linker)
def go_entity_linker(document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

In [0]:
linker = EntityLinker(k = 10,max_entities_per_mention = 2, name='hpo')  #parameters are tunable,so it can be set to return more than 2 entity matches
nlp = en_ner_bc5dr_md.load()
nlp.add_pipe(linker)
def hpo_entity_linker(document):
    """ A function that accepts the name of a scispacy knowledge base and documents and returns the entity link details"""
    #linker = EntityLinker(k = 10,max_entities_per_mention = 2, name=linker_name)  #parameters are tunable,so it can be set to return more than 2 entity matches
    #nlp = en_core_sci_sm.load()
    #nlp.add_pipe(linker)
    doc = nlp(document)
    try:
        entity = doc.ents[0]
    except IndexError:
        entity = 'Nan'
    entity_details = []
    entity_details.append(entity)
    try:
        for linker_ent in entity._.kb_ents:
            Concept_Id, Score = linker_ent
            entity_details.append('Entity_Matching_Score :{}'.format(Score))
            entity_details.append(linker.kb.cui_to_entity[linker_ent[0]])
    except AttributeError:
        pass
    return entity_details

entities_and_label_from_4_NER_model_dataframe['umls_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : umls_entity_linker(x))

entities_and_label_from_4_NER_model_dataframe['mesh_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : mesh_entity_linker(x))

entities_and_label_from_4_NER_model_dataframe['go_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : go_entity_linker(x))

entities_and_label_from_4_NER_model_dataframe['hpo_output'] = entities_and_label_from_4_NER_model_dataframe['Entity'].swifter.apply(lambda x : hpo_entity_linker(x))

pd.options.display.max_colwidth = 1000      #increase display width of the pandas dataframe
entities_and_label_from_4_NER_model_dataframe.head()